---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [37]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
#import matplotlib.pyplot as plt
#%matplotlib notebook

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [9]:
def graph_identification():
    
    avg_cls = [nx.average_clustering(i) for i in P1_Graphs]
    avg_spl = [nx.average_shortest_path_length(i) for i in P1_Graphs]

    answer =[]
    for i in range(len(P1_Graphs)):
        degrees = P1_Graphs[i].degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(j)/float(nx.number_of_nodes(P1_Graphs[i])) for j in degree_values]
        if len(histogram) > 10:
            answer.append('PA')
        elif avg_cls[i] <= 0.1:
            answer.append('SW_H')
        else:
            answer.append('SW_L')
    
    return answer

In [10]:
#graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

In [8]:
#avg_cls = [nx.average_clustering(i) for i in P1_Graphs]
#avg_cls

[0.03167539146454044,
 0.5642419635919628,
 0.4018222222222227,
 0.03780379975223251,
 0.0033037037037037037]

In [11]:
#avg_spl = [nx.average_shortest_path_length(i) for i in P1_Graphs]
#avg_spl

[4.099161161161161,
 5.089871871871872,
 9.378702269692925,
 3.1048046283934134,
 5.0785509568313305]

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [15]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


In [22]:
G.nodes(data=True)[:10]

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan}),
 (3, {'Department': 21, 'ManagementSalary': 1.0}),
 (4, {'Department': 21, 'ManagementSalary': 1.0}),
 (5, {'Department': 25, 'ManagementSalary': nan}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (7, {'Department': 14, 'ManagementSalary': 0.0}),
 (8, {'Department': 14, 'ManagementSalary': nan}),
 (9, {'Department': 14, 'ManagementSalary': 0.0})]

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [58]:
def salary_predictions():
    
    #creating the df with the applicable features
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['Degree'] = pd.Series(G.degree())
    df['Closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['PageRank'] = pd.Series(nx.pagerank(G))
    df['Degree_Centrality'] = pd.Series(nx.degree_centrality(G))
    
    #manual train/test split
    Train_df = df[~pd.isnull(df['ManagementSalary'])]
    Test_df = df[pd.isnull(df['ManagementSalary'])]
    features =['Clustering', 'Betweeness', 'Degree',
               'Closeness', 'PageRank', 'Degree_Centrality']
    X_train = Train_df[features]
    y_train = Train_df['ManagementSalary']
    X_test = Test_df[features]
    
    #using scaling to normalize the data
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    #using a random forest as the classifier worked best out of the classifiers tried
    clf = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X_train_scaled, y_train)
    predictions = (clf.predict_proba(X_test_scaled)[:,1]).tolist()
    answer = pd.Series(data=predictions,index=X_test.index)
    
    return answer

In [60]:
#salary_predictions()[0:10]

1     0.012852
2     0.930000
5     1.000000
8     0.322301
14    0.053968
18    0.075302
27    0.024921
30    0.791955
31    0.497149
34    0.057000
dtype: float64

In [31]:
#df.head()

,Department,ManagementSalary,Clustering,Betweeness,Degree,Closeness,PageRank,Degree_Centrality
0,1,0.0,0.276423,0.001124,44,0.421991,0.001224,0.043825
1,1,NaN,0.265306,0.001195,52,0.422360,0.001426,0.051793
2,21,NaN,0.297803,0.006570,95,0.461490,0.002605,0.094622
3,21,1.0,0.384910,0.001654,71,0.441663,0.001833,0.070717
4,21,1.0,0.318691,0.005547,96,0.462152,0.002526,0.095618


In [33]:
#Train_df = df[~pd.isnull(df['ManagementSalary'])]
#Train_df.head()

,Department,ManagementSalary,Clustering,Betweeness,Degree,Closeness,PageRank,Degree_Centrality
0,1,0.0,0.276423,0.001124,44,0.421991,0.001224,0.043825
3,21,1.0,0.384910,0.001654,71,0.441663,0.001833,0.070717
4,21,1.0,0.318691,0.005547,96,0.462152,0.002526,0.095618
6,25,1.0,0.155183,0.012387,115,0.475805,0.003146,0.114542
7,14,0.0,0.287785,0.002818,72,0.420156,0.002002,0.071713


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [68]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [70]:
def new_connections_predictions():
    
    # set up dataframe with features
    future_connections['Preferential Attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['Common Neighbours'] = [len(list(nx.common_neighbors(G, x[0], x[1]))) for x in future_connections.index]
    future_connections['Jaccard Coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['Resource Allocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    future_connections['Adamic Adar'] = [i[2] for i in nx.adamic_adar_index(G, future_connections.index)]
    
    #assign community inclusivity
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
        
    #add in the features that required community inclusion
    future_connections['cn_soundarajan_hopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, future_connections.index)]   
    future_connections['ra_soundarajan_hopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, future_connections.index)]
    
    #run the random forest classifier just like in the last question
    Train_df = future_connections[~pd.isnull(future_connections['Future Connection'])]
    Test_df = future_connections[pd.isnull(future_connections['Future Connection'])]
    features =['Preferential Attachment', 'Common Neighbours', 'Jaccard Coefficient', 'Resource Allocation', 'Adamic Adar',
           'cn_soundarajan_hopcroft', 'ra_soundarajan_hopcroft']
    X_train = Train_df[features]
    y_train = Train_df['Future Connection']
    X_test = Test_df[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    clf = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X_train_scaled, y_train)
    predictions = (clf.predict_proba(X_test_scaled)[:,1]).tolist()
    answer = pd.Series(predictions,X_test.index)
    
    return answer

In [73]:
#new_connections_predictions()

(107, 348)    0.026768
(542, 751)    0.012583
(20, 426)     0.588224
(50, 989)     0.012571
(942, 986)    0.012839
(324, 857)    0.012717
(13, 710)     0.121900
(19, 271)     0.137943
(319, 878)    0.012663
(659, 707)    0.012655
(49, 843)     0.012494
(208, 893)    0.012667
(377, 469)    0.013119
(405, 999)    0.015039
(129, 740)    0.013256
(292, 618)    0.018907
(239, 689)    0.012367
(359, 373)    0.016161
(53, 523)     0.428777
(276, 984)    0.012405
(202, 997)    0.012380
(604, 619)    0.034297
(270, 911)    0.012516
(261, 481)    0.061996
(200, 450)    0.997601
(213, 634)    0.012512
(644, 735)    0.039528
(346, 553)    0.012391
(521, 738)    0.012022
(422, 953)    0.013935
                ...   
(672, 848)    0.012516
(28, 127)     0.987940
(202, 661)    0.012370
(54, 195)     0.999871
(295, 864)    0.012717
(814, 936)    0.012512
(839, 874)    0.012839
(139, 843)    0.012717
(461, 544)    0.013347
(68, 487)     0.013216
(622, 932)    0.012713
(504, 936)    0.018105
(479, 528) 

In [74]:
#future_connections.head(10)

,Future Connection,Preferential Attachment,Common Neighbours,Jaccard Coefficient,Resource Allocation,Adamic Adar,cn_soundarajan_hopcroft,ra_soundarajan_hopcroft
"(6, 840)",0.0,2070,9,0.073770,0.136721,2.110314,9,0.0
"(4, 197)",0.0,3552,2,0.015504,0.008437,0.363528,2,0.0
"(620, 979)",0.0,28,0,0.000000,0.000000,0.000000,0,0.0
"(519, 872)",0.0,299,2,0.060606,0.039726,0.507553,2,0.0
"(382, 423)",0.0,205,0,0.000000,0.000000,0.000000,0,0.0
"(97, 226)",1.0,1575,4,0.048193,0.036296,0.843588,4,0.0
"(349, 905)",0.0,240,0,0.000000,0.000000,0.000000,0,0.0
"(429, 860)",0.0,816,0,0.000000,0.000000,0.000000,0,0.0
"(309, 989)",0.0,184,0,0.000000,0.000000,0.000000,0,0.0
"(468, 880)",0.0,672,1,0.019608,0.002882,0.170960,1,0.0
